# Merge the controlled values

This notebook provides the following functionality:
* The controlled data should be added in existing empty columns, with the same name as the controlled vocabulary (e.g. invasionStage)
* The mapping should be done on dataset= datasetName, verbatimValue = e.g verbatimInvasionStage
* It would be nice to get errors if a mapping cannot be found for non-blank values...

In [27]:
import numpy as np
import pandas as pd

In [28]:
concatenated = pd.read_csv('../data/processed/checklist.tsv', delimiter='\t', dtype=object)

### Single column mapping

Writing a function for the execution of the single column mapping

In [56]:
def merge_control_vocabulary(concatenated, mapping, mappingname):
    """combine the controlled mapping with the names provided in the current version of the checklist
    """
    merge_column = ''.join(['verbatim', ''.join([mappingname[0].upper(), mappingname[1:]])])
    merged = pd.merge(concatenated[["datasetName", merge_column]], mapping, how='left', 
         left_on=["datasetName", merge_column], right_on=['dataset', 'verbatimValue'])
    concatenated[mappingname] = merged["controlledValue"]
    return concatenated

List with the mappings that need to be done
(cfr. https://github.com/inbo/alien-species-checklist/issues/64):

In [57]:
mappings = ['invasionStage', 'introductionPathway', 'habitat', 'nativeRange']

Loop over the mapping and updating the concatenated file:

In [58]:
for term in mappings:
    print(term)
    mapping = pd.read_csv(''.join(["../data/vocabularies/", term, "-mapping.tsv"]), delimiter='\t')
    concatenated = merge_control_vocabulary(concatenated, mapping, term)

invasionStage
introductionPathway
habitat
nativeRange


### Multi-column mapping

When N (> 1) columns need to be mapped together, the join should take into account the combined set of names:

In [36]:
term = 'presence'

In [37]:
mapping = pd.read_csv(''.join(["../data/vocabularies/", term, "-mapping.tsv"]), delimiter='\t')

In [38]:
merge_columns = [term for term in mapping.columns if 'verbatim' in term]

In [39]:
merge_columns_mapping = ["dataset"] + merge_columns
merge_columns_concat = ["datasetName"] + merge_columns

Merging for the entire set of columns available in the mapping:

In [46]:
merged = pd.merge(concatenated[merge_columns_concat], mapping, how='left', 
     left_on=merge_columns_concat, right_on=merge_columns_mapping)


In [47]:
merge_columns_final = [''.join([term[8].lower(), term[9:]]) for term in merge_columns]
concatenated[merge_columns_final] = merged[merge_columns_final]

### Saving the concatenated

In [55]:
#concatenated.to_csv('../data/processed/checklist.tsv', sep='\t', dtype=object)

### Checkup

Check if a mapping cannot be found for non-blank values:

Single column mappings:

In [59]:
for term in mappings:
    number_of_unmapped = sum(concatenated['verbatimNativeRange'].notnull() & concatenated['nativeRange'].isnull())
    if number_of_unmapped == 0:
        message = 'Great!'
    else:
        message = 'Bummer'        
    print(term, "has", str(number_of_unmapped), " badly mapped values. ", message)

invasionStage has 0  badly mapped values.  Great!
introductionPathway has 0  badly mapped values.  Great!
habitat has 0  badly mapped values.  Great!
nativeRange has 0  badly mapped values.  Great!


Multi column mapping

In [60]:
merge_columns_mapping

['dataset',
 'verbatimPresenceBE',
 'verbatimPresenceFL',
 'verbatimPresenceWA',
 'verbatimPresenceBR',
 'verbatimPresenceBECoast',
 'verbatimPresenceBPNS',
 'verbatimPresenceBEEZ']

In [61]:
sum(concatenated[merge_columns].notnull().all(axis=1) & concatenated[merge_columns_final].isnull().all(axis=1))

0

(if 0, mapping is succesfull)